In [28]:
!pip install openai

In [44]:
import openai
from getpass import getpass
from openai import OpenAI
import json
import pandas as pd
import requests

In [30]:
api_key = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [31]:
client = OpenAI(api_key = api_key)

def ask_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4o"
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant for career and compensation guidance."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [32]:
user_question = "What skills can help me double my salary?"
response = ask_gpt(user_question)
print("💬 AI Assistant:", response)

💬 AI Assistant: Doubling your salary typically requires a combination of valuable skills, experience, and strategic career planning. Here are some skills that can help you increase your earning potential:

1. **Technical Skills**: Acquiring in-demand technical skills related to your industry can significantly boost your salary. This could include programming languages, data analysis skills, digital marketing expertise, or specialized technical knowledge.

2. **Leadership Skills**: Developing strong leadership skills such as decision-making, strategic thinking, team management, and project management can position you for higher-paying leadership roles.

3. **Communication Skills**: Effective communication skills are crucial in almost every profession. Being able to articulate your ideas clearly, negotiate effectively, and build strong relationships with clients and colleagues can lead to salary advancements.

4. **Negotiation Skills**: Learning to negotiate your salary and benefits can 

In [33]:
def classify_intent(user_input):
    input_lower = user_input.lower()

    if "double my salary" in input_lower or "increase salary" in input_lower:
        return "salary_growth"
    elif "compare" in input_lower or "industry benchmark" in input_lower:
        return "compensation_comparison"
    elif "learn" in input_lower or "skills" in input_lower:
        return "skill_recommendation"
    elif "what if" in input_lower or "switch to" in input_lower:
        return "what_if_simulation"
    else:
        return "general"

In [34]:
def get_prompt(intent, user_input):
    if intent == "salary_growth":
        return f"User asked: '{user_input}'. Give clear, data-based ways to significantly increase their salary."

    elif intent == "compensation_comparison":
        return f"User asked: '{user_input}'. Compare their compensation to industry benchmarks using available data."

    elif intent == "skill_recommendation":
        return f"User asked: '{user_input}'. Suggest specific, high-impact skills they can learn to earn more."

    elif intent == "what_if_simulation":
        return f"User asked: '{user_input}'. Simulate the effect of that career move. Estimate how their compensation might change."

    else:
        return f"Answer this career-related question in a helpful, structured way: '{user_input}'"

In [41]:
def extract_user_info(user_input):
    extraction_prompt = f"""
Extract the following fields from this user input:
- Job title (just the title, no extra words)
- Salary (as a number, no $ or commas)
- Location (if mentioned)

User Input: \"{user_input}\"

Respond ONLY in valid JSON like this:
{{"job_title": "...", "salary": ..., "location": "..."}}
If a field is missing, set its value to null.
"""
    response = ask_gpt(extraction_prompt)

    # Try to parse JSON safely
    try:
        parsed = json.loads(response)
    except json.JSONDecodeError:
        return {"job_title": None, "salary": None, "location": None}

    return parsed


In [52]:
job_title_to_series_id = {
    "software developer": "CEU3100000001",  # OEWS for Software Developers (SOC 15-1252)
    "data scientist": "OEUN000000151255",      # Example: change as needed
    "accountant": "OEUN000000132011",          # Accountants and Auditors (SOC 13-2011)
    "registered nurse": "OEUN000000291141",    # SOC 29-1141
}

In [48]:
def get_series_id(job_title):
    return job_title_to_series_id.get(job_title.lower())


In [49]:
def get_latest_salary_value(series_id):
    data = get_bls_salary_data(series_id)

    try:
        series = data['Results']['series'][0]['data']
        latest_entry = series[0]  # Most recent month
        return latest_entry['value'], latest_entry['periodName'], latest_entry['year']
    except Exception as e:
        print("Error parsing BLS response:", e)
        return None

In [50]:
def get_salary_fact(job_title):
    series_id = get_series_id(job_title)
    if not series_id:
        return f"Sorry, I don't have wage data for '{job_title}'."

    result = get_latest_salary_value(series_id)
    if not result:
        return f"Sorry, couldn't fetch salary data for '{job_title}'."

    value, month, year = result
    return f"As of {month} {year}, the average wage for a {job_title} was ${value} annually (BLS data)."

In [53]:
print(get_salary_fact("software developer"))

Error parsing BLS response: 'series'
Sorry, couldn't fetch salary data for 'software developer'.


In [46]:
bls_api_key = "1f0edec30fe680b494b95fc3e644c613d1ac5327de84bd691a0efbdffbdd5747"

def get_bls_salary_data(series_id):
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    headers = {"Content-type": "application/json"}

    data = {
        "registrationKey": bls_api_key,
        "seriesid": [series_id],
        "startyear": "2022",
        "endyear": "2023"
    }

    response = requests.post(url, json=data, headers=headers)
    return response.json()

In [39]:
def handle_user_query(user_input):
    intent = classify_intent(user_input)

    if intent == "compensation_comparison":
        info = extract_user_info(user_input)
        job_title = info.get("job_title")
        user_salary = info.get("salary")

        if not job_title or not user_salary:
            return "Sorry, I couldn't extract your job title and salary. Please include them in your question."

        series_id = get_series_id(job_title)
        if not series_id:
            return f"Sorry, I don't have data for the job title '{job_title}' yet."

        fact = build_comparison_fact(job_title, user_salary, series_id)
        prompt = f"{fact}\n\nExplain this in plain English and suggest next steps to improve salary."
        return ask_gpt(prompt)

    elif intent == "salary_estimation":
        info = extract_estimation_info(user_input)
        if not info:
            return "Sorry, I couldn’t understand enough details. Please mention your job, skills, and experience."

        est = estimate_salary(
            job_title=info.get("job_title"),
            location=info.get("location"),
            experience_years=info.get("experience_years", 0),
            user_skills=info.get("skills", []),
            education=info.get("education", "")
        )

        if est is None:
            return f"Sorry, I don't have base data for {info.get('job_title')} in {info.get('location')}."

        explanation = explain_salary_estimate(est, info.get("job_title"), info.get("skills", []))
        return explanation

    # Fallback for everything else
    prompt = get_prompt(intent, user_input)
    return ask_gpt(prompt)


In [55]:
question = "What if I switch to data science?"
print("🧠 Assistant:", handle_user_query(question))

🧠 Assistant: Switching to a career in data science can have a significant impact on your compensation, as data scientists are in high demand due to the increasing importance of data-driven decision-making in various industries. Here's a general estimation of how your compensation might change:

1. **Entry-Level Data Scientist:** If you are starting out as an entry-level data scientist, you can expect to earn a salary ranging from $60,000 to $90,000 per year, depending on your location, experience, and the company you work for.

2. **Mid-Level Data Scientist:** As you gain more experience and expertise in data science, your salary could range from $90,000 to $130,000 per year. At this level, you may also be eligible for bonuses and additional perks based on your performance.

3. **Senior Data Scientist:** With several years of experience and a track record of delivering impactful insights, you could earn a salary of $130,000 to $180,000 per year as a senior data scientist. Senior data s